In [80]:
!rm sql/hawaii.sqlite

In [81]:
import pandas as pd
# Import SQL Alchemy
import sqlalchemy
from sqlalchemy.sql import func
from sqlalchemy.orm import Session

# Import and establish Base for which classes will be constructed 
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float, Text

In [82]:
engine = sqlalchemy.create_engine("sqlite:///sql/hawaii.sqlite")
session = Session(bind=engine)

In [83]:
prcp_df = pd.read_csv("clean_resources/hawaii_prcp_measurements.csv")
#can drop "Unnamed: 0" instead of setting the index if a continuous index is necessary
prcp_df = prcp_df.rename(columns = {"Unnamed: 0": "measurement_id", "station": "station_id"})#.set_index("measurement_id")
prcp_df.head()

,measurement_id,station_id,date,prcp,tobs
0,0,USC00519397,2010-01-01,0.08,65
1,1,USC00519397,2010-01-02,0.00,63
2,2,USC00519397,2010-01-03,0.00,74
3,3,USC00519397,2010-01-04,0.00,76
4,5,USC00519397,2010-01-07,0.06,70


In [84]:
temp_df = pd.read_csv("clean_resources/hawaii_temp_measurements.csv")
#can drop "Unnamed: 0" instead of setting the index if a continuous index is necessary
temp_df = temp_df.rename(columns = {"Unnamed: 0": "measurement_id", "station": "station_id"})#.set_index("measurement_id")
temp_df.head()

,measurement_id,station_id,date,tobs
0,0,USC00519397,2010-01-01,65
1,1,USC00519397,2010-01-02,63
2,2,USC00519397,2010-01-03,74
3,3,USC00519397,2010-01-04,76
4,4,USC00519397,2010-01-06,73


In [85]:
stations_df = pd.read_csv("clean_resources/hawaii_stations.csv")
stations_df = stations_df.rename(columns = {"station": "station_id"}).drop('Unnamed: 0', 1)#.set_index("station_id")
stations_df

,station_id,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.27160,-157.81680,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.42340,-157.80150,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.52130,-157.83740,7.0
3,USC00517948,"PEARL CITY, HI US",21.39340,-157.97510,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.49920,-158.01110,306.6
5,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",21.33556,-157.71139,19.5
6,USC00519281,"WAIHEE 837.5, HI US",21.45167,-157.84889,32.9
7,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",21.31520,-157.99920,0.9
8,USC00516128,"MANOA LYON ARBO 785.2, HI US",21.33310,-157.80250,152.4


In [86]:
conn = engine.connect()

In [87]:
class Measurement(Base):
    __tablename__ = "prcp_measurements"
    
    measurement_id = Column(Integer, primary_key = True)
    station_id = Column(Text)#, ForeignKey("stations.station_id")),
    date = Column(Text)
    prcp = Column(Float)
    tobs = Column(Float)
    
    def __repr__(self):
        return f"id={self.measurement_id}, prcp={self.prcp}"

class Temp_Measurement(Base):
    __tablename__ = "temp_measurements"
    
    measurement_id = Column(Integer, primary_key = True)
    station_id = Column(Text)#, ForeignKey("stations.station_id")),
    date = Column(Text)
    tobs = Column(Float)
    
    def __repr__(self):
        return f"id={self.measurement_id}, temp={self.tobs}"

In [88]:
class Station(Base):
    __tablename__ = "stations"
    
    station_id = Column(String, primary_key = True)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    
    def __repr__(self):
        return f"id={self.station_id}, name={self.name}"

In [89]:
Base.metadata.create_all(engine)


In [90]:
#prcp_df.to_sql("prcp_measurements",engine)
#temp_df.to_sql("temp_measurements",engine)
#stations_df.to_sql("stations",engine)

In [91]:
prcp_data = prcp_df.to_dict(orient='records')
temp_data = temp_df.to_dict(orient='records')
stations_data = stations_df.to_dict(orient='records')

In [94]:
stations_data

[{'elevation': 3.0,
  'latitude': 21.2716,
  'longitude': -157.8168,
  'name': 'WAIKIKI 717.2, HI US',
  'station_id': 'USC00519397'},
 {'elevation': 14.6,
  'latitude': 21.4234,
  'longitude': -157.8015,
  'name': 'KANEOHE 838.1, HI US',
  'station_id': 'USC00513117'},
 {'elevation': 7.0,
  'latitude': 21.5213,
  'longitude': -157.8374,
  'name': 'KUALOA RANCH HEADQUARTERS 886.9, HI US',
  'station_id': 'USC00514830'},
 {'elevation': 11.9,
  'latitude': 21.3934,
  'longitude': -157.9751,
  'name': 'PEARL CITY, HI US',
  'station_id': 'USC00517948'},
 {'elevation': 306.6,
  'latitude': 21.4992,
  'longitude': -158.0111,
  'name': 'UPPER WAHIAWA 874.3, HI US',
  'station_id': 'USC00518838'},
 {'elevation': 19.5,
  'latitude': 21.33556,
  'longitude': -157.71139,
  'name': 'WAIMANALO EXPERIMENTAL FARM, HI US',
  'station_id': 'USC00519523'},
 {'elevation': 32.9,
  'latitude': 21.45167,
  'longitude': -157.84888999999995,
  'name': 'WAIHEE 837.5, HI US',
  'station_id': 'USC00519281'},
 {

In [95]:

metadata = sqlalchemy.MetaData(bind=engine)
metadata.reflect()

In [96]:
prcp_table = sqlalchemy.Table("prcp_measurements", metadata, autoload = True )
temp_table = sqlalchemy.Table("temp_measurements", metadata, autoload = True )
stations_table = sqlalchemy.Table("stations", metadata, autoload = True )


In [97]:
conn.execute(prcp_table.insert(), prcp_data)
conn.execute(temp_table.insert(), temp_data)
conn.execute(stations_table.insert(), stations_data)

In [98]:
len(conn.execute("select * from prcp_measurements").fetchall())

18103